In [ ]:
import inspect, sys, pathlib
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Make project root importable
root = pathlib.Path('..', '..').resolve()
sys.path.append(str(root))

from quasar.planner import Planner
from quasar.simulation_engine import SimulationEngine
from benchmarks.circuits import ghz_circuit, grover_circuit


In [ ]:
alphas = [1, 2, 4, 8]
circuits = {
    'GHZ_8': ghz_circuit(8),
    'Grover_5': grover_circuit(5, 1),
}
records = []
for name, circ in circuits.items():
    for alpha in alphas:
        kwargs = {'conversion_cost_multiplier': alpha}
        if 'compare_pre_pass_costs' in inspect.signature(Planner).parameters:
            kwargs['compare_pre_pass_costs'] = True
        planner = Planner(**kwargs)
        engine = SimulationEngine(planner=planner)
        plan = planner.plan(circ)
        records.append({'circuit': name, 'alpha': alpha, 'backend': plan.final_backend.name})
df = pd.DataFrame(records)
pivot = df.pivot('circuit', 'alpha', 'backend')
sns.heatmap(pivot, annot=True, fmt='', cmap='viridis')
plt.title('Planner backend choice')
plt.show()
results = df


In [ ]:
import json, pathlib
try:
    import ipynbname
    nb_name = ipynbname.path().stem
except Exception:  # pragma: no cover
    nb_name = 'plan_choice_heatmap'
_params = {k: v for k, v in globals().items() if not k.startswith('_') and isinstance(v, (int, float, str, bool, list, dict, tuple))}
pathlib.Path('../results').mkdir(exist_ok=True)
with open(f"../results/{nb_name}_params.json", 'w') as f:
    json.dump(_params, f, indent=2)
with open(f"../results/{nb_name}_results.json", 'w') as f:
    json.dump(results.to_dict(orient='records'), f, indent=2)
print(json.dumps(_params, indent=2))
